In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import sklearn
import timescaledb_model as tsdb
from analyze import get_files_infos_df, store_file
from tqdm import tqdm 
from utils import multi_read_df_from_paths

Logs of timescaledb_model go to /tmp/bourse.log


In [3]:
# files_infos_df = get_files_infos_df()
# # files_infos_df.to_pickle('files_infos.pkl'

## Files Dataframe - Filtering
1 hour for each Last day of Each month

In [121]:
files_infos_df = get_files_infos_df("./files_infos.pkl")

In [141]:
np.random.seed(10)
dates_to_take = files_infos_df.groupby([files_infos_df["year_month"]]).apply(lambda x :x.sample(n=1)).reset_index(drop=True)["date"]
file_infos_df_filtered = files_infos_df[files_infos_df["date"].isin(dates_to_take)]

In [142]:
file_infos_df_filtered["hour"] = file_infos_df_filtered.index.hour
file_infos_df_filtered = file_infos_df_filtered.groupby([file_infos_df_filtered["date"], file_infos_df_filtered["hour"], file_infos_df_filtered["prefix"]]).first().reset_index()

In [144]:
dfs = multi_read_df_from_paths(list(file_infos_df_filtered["path"]))

100%|██████████| 1944/1944 [00:07<00:00, 271.48it/s]


## PEA PME - Symbol NF handling

In [137]:
df =store_file(files_infos_df[files_infos_df["prefix"] == "peapme"]["path"].iloc[30])

In [138]:
df["prefix"] = df["symbol"].apply(lambda x: x[0:3])

In [139]:
df["prefix"].value_counts()

prefix
1rP    460
1rE     35
FF1     13
1rA      2
Name: count, dtype: int64

In [41]:
map_prefix_to_symbol_nf = {
    "1rP": lambda x: x[3:8],
    "1rA": lambda x: x[3:8],
    "1rE": lambda x: x[4:9],
    "FF1": lambda x: x.split("_")[1][0:5],
}

df["symbol_nf"] = df.apply(
    lambda x: map_prefix_to_symbol_nf.get(x["prefix"], lambda _: np.nan)(x["symbol"]),
    axis=1,
)

## CompA - Symbol NF handling